<a href="https://colab.research.google.com/github/idkhoa1605/CSDL-HCMUS/blob/main/PrivateMulticativeWeightedMajority.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Câu 2: Mô phỏng học trực tuyến
MSSV: 18120040

Họ tên: Nguyễn Đăng Khoa



## Tiền xử lí 


### Chuẩn bị dữ liệu
Trong bài làm này, nhóm em sử dụng bộ dữ liệu giá nhà ở California. Tải bộ dữ liệu [tại đây](https://drive.google.com/file/d/1-nFIMko7L5erYkmKk-RqjsBV-jADpIeS/view?usp=sharing)
Ta sẽ dự đoán `median_house_value` bằng cách sử dụng các đặc trưng khác

In [1]:
# Cài đặt thư viện cần thiết
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
df = pd.read_csv('housing.csv')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [2]:
# Tiền xử lý: Loại bỏ cột longtitude, latitude, encode cột ocean_proximity
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(df[['ocean_proximity']])
df['ocean_encoded'] = housing_cat_encoded+1

housing = df.drop(["latitude", "longitude","ocean_proximity"], axis=1)
median = housing["total_bedrooms"].median()  # option 3
housing["total_bedrooms"].fillna(median, inplace=True)

In [3]:
# Chia thành 3 tập con D1, D2, D3 theo tỷ lệ 4:4:2
rand_indices = np.random.permutation(len(df))

D_trained_size = int(len(df) * 0.4)
D1 = housing.iloc[rand_indices[:D_trained_size]].reset_index()
D2 = housing.iloc[rand_indices[D_trained_size:D_trained_size * 2]].reset_index()
D3 =  housing.iloc[rand_indices[: int(D_trained_size / 2)]].reset_index()

In [4]:
# Chia tập train và test cho 3 bộ dữ liệu
X1 = D1.copy().drop(["index","median_house_value"], axis= 1)
y1 = D1.loc[:, "median_house_value"]
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X1, y1, test_size=0.2, random_state=42)


X2 = D2.copy().drop(["index","median_house_value"], axis= 1)
y2 = D2.loc[:, "median_house_value"]
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X2, y2, test_size=0.2, random_state=42)


X3 = D3.copy().drop(["index","median_house_value"], axis= 1)
y3 = D3.loc[:, "median_house_value"]
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X3, y3, test_size=0.2, random_state=42)

## Huấn luyện và đánh giá cả 3 mô hình

In [5]:
# Huấn luyện 3 mô hình S_1, S_2, S_3
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

S_1 = LinearRegression()
S_1.fit(X_train_1, y_train_1)
y_hat_1 = S_1.predict(X_test_1)
MSE_1 = mean_squared_error(y_test_1, y_hat_1, squared=False)

S_2 = LinearRegression()
S_2.fit(X_train_2, y_train_2)
y_hat_2 = S_2.predict(X_test_2)
MSE_2 = mean_squared_error(y_test_2, y_hat_2, squared=False)

S_model = LinearRegression()
S_model.fit(X_train_3, y_train_3)
y_hat_3 = S_model.predict(X_test_3)
MSE_3 = mean_squared_error(y_test_3, y_hat_3, squared=False)

## Thuật giải đồng thuận


In [6]:
import warnings
warnings.filterwarnings("ignore")

import random

### Thuat giai dong thuan su dung co che Luy thua
#   @parameter:
#       - model: np.array([LinerRegression(), ...]) - mang cac mo hinh 
#       - data: np.array([X_train, y_train]) - Bo du lieu gui di cho cac mo hinh
#       - target_model: LinerRegression.coef_  - tham so cua mo hinh can hoc
#       - T: int - so luong vong can thuc hien
#       - epsilon: float - Ngan sach bao mat
#       - theta: float - Do chinh xac
#   @Output:
#       - Regret: np.array(k) - Mang trung binh mat mat cua cac thuat giai A tren T vong


def PrivateMVM(models, data, target_model, T = 10, epsilon = 1.0, theta= 0.8):
  # Tinh cac tham so can thiet
  learning_rate = epsilon / np.sqrt(32 * T * np.log(1 / theta))
  n = len(data[0])
  k = len(models)
  l_t = np.zeros(k)

  # Khoi tao trong so cho tung chuyen gia
  W = np.ones(k)
  for _ in range(T):
    weights_model = W / np.sum(W)
    picked = np.array(random.choices(list(enumerate(models)), weights = weights_model, k = n))
    print(target_model)
    for x in range(n):
      i = picked[x][0]
      l_i_t = picked[x][1].predict([data[0][x]]) - data[1][x]
      target_model -= learning_rate * data[0][x] * l_i_t
      W[i] = W[i] * np.exp(-learning_rate * l_i_t)
      l_t[i] += l_i_t

  l_t /= T
  l_t -= np.amin(l_t)
  return l_t



In [7]:
# Chuẩn bị dữ liệu để chạy thuật giải
experts = [S_1, S_2]
train_data = []
train_data.append(np.array(X_train_3))
train_data.append(np.array(y_train_3))

In [8]:
Regret = PrivateMVM(experts, train_data, S_model.coef_, 20)

[ 1.90072704e+03 -2.58694919e+01  1.12060028e+02 -2.30354334e+01
  1.19132052e+02  5.03282895e+04  4.94273652e+03]
[ 4.70562336e+06  1.60686465e+09  3.85079161e+08  2.35864969e+09
  3.70245554e+08  1.82972528e+06 -2.76689632e+05]
[ 5.46444935e+06  3.38003252e+09  8.13868673e+08  4.87199327e+09
  7.63511940e+08  3.05411374e+06 -6.77170820e+05]
[ 6.22327534e+06  5.15320039e+09  1.24265818e+09  7.38533685e+09
  1.15677833e+09  4.27850219e+06 -1.07765201e+06]
[ 6.98210133e+06  6.92636826e+09  1.67144770e+09  9.89868043e+09
  1.55004471e+09  5.50289065e+06 -1.47813319e+06]
[ 7.74092732e+06  8.69953613e+09  2.10023721e+09  1.24120240e+10
  1.94331110e+09  6.72727910e+06 -1.87861438e+06]
[ 8.49975331e+06  1.04727040e+10  2.52902672e+09  1.49253676e+10
  2.33657749e+09  7.95166756e+06 -2.27909557e+06]
[ 9.25857930e+06  1.22458719e+10  2.95781623e+09  1.74387112e+10
  2.72984387e+09  9.17605601e+06 -2.67957676e+06]
[ 1.00174053e+07  1.40190397e+10  3.38660574e+09  1.99520547e+10
  3.12311026e+0

In [9]:
# Chạy thử mô hình
y_hat_3_1 = S_model.predict(X_test_3)
MSE_4 = mean_squared_error(y_test_3, y_hat_3_1, squared=False)
MSE_4

200400766655478.62

In [10]:
Regret

array([1069007.55793102,       0.        ])